In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import scale
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import make_scorer


from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE

import seaborn as sns

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
#Ideas:
#Use a custom objective function (XGBoost) to maximize profits directly 
#i.e. not caring about the accuracy of the predictions

#Change odds from H D A to sum H/D H/A
#Try it you will like it.

In [5]:
#Project steps:
#1
#Extract relevant features from football-data.co.uk. Do some feature engineering.
#Expect some improvement from competence_difference. Better formula may help
#Do we need more data?

#2
#Predict outcomes of each match. Training using data since 2009. Leave 2015 and 2016.
#2015 is for cross-validation of classifiers. 2016 is for testing 

#3
#Estimate probability that prediction is correct. In other words, how confident are we?

#4 
#Using all the information gathered, formulate a betting strategy

In [8]:
f9 = r'/home/william/Dropbox/Kaggle/Soccer/season9.csv'
f10 = r'/home/william/Dropbox/Kaggle/Soccer/season10.csv'
f11 = r'/home/william/Dropbox/Kaggle/Soccer/season11.csv'
f12 = r'/home/william/Dropbox/Kaggle/Soccer/season12.csv'
f13 = r'/home/william/Dropbox/Kaggle/Soccer/season13.csv'
f14 = r'/home/william/Dropbox/Kaggle/Soccer/season14.csv'
f15 = r'/home/william/Dropbox/Kaggle/Soccer/season15.csv'
f16 = r'/home/william/Dropbox/Kaggle/Soccer/season16.csv'


df9 = pd.read_csv(f9)
df10 = pd.read_csv(f10)
df11 = pd.read_csv(f11)
df12 = pd.read_csv(f12)
df13 = pd.read_csv(f13)
df14 = pd.read_csv(f14)
df15 = pd.read_csv(f15)
df16 = pd.read_csv(f16)
df_test = df16

df_train = df9.append([df10,df11,df12,df13,df14,df15],ignore_index=True)
#df_train['sum_odds'] = df_train['B365H']+df_train['B365A']+df_train['B365D']
#df_train['H'] = df_train['B365H']/df_train['B365D']
#df_train['A'] = df_train['B365A']/df_train['B365D']
#df_train = df_train.drop(['B365H', 'B365D', 'B365A'],axis=1)

f_training = r'/home/william/Dropbox/Kaggle/Soccer/training.csv'
f_testing = r'/home/william/Dropbox/Kaggle/Soccer/testing.csv'

#df_train = pd.read_csv(f_training)
#df_test = pd.read_csv(f_testing)

#df_train = df_train.drop('B365A',axis=1)
#df_test = df_test.drop('B365A',axis=1)


train_X = df_train.drop('Home/Away win',axis=1)
train_y = df_train['Home/Away win']


#df16['sum_odds'] = df16['B365H']+df16['B365A']+df16['B365D']
#df16['H'] = df16['B365H']/df16['B365D']
#df16['A'] = df16['B365A']/df16['B365D']
#df16 = df16.drop(['B365H', 'B365D', 'B365A'],axis=1)



test_X = df_test.drop('Home/Away win',axis=1)
test_y = df_test['Home/Away win']





#Adding a feature to indicate extra likelihood of draw
train_draw = np.array((train_y==0)+0)
test_draw = np.array((test_y==0)+0)
draw_logit = GaussianNB()
draw_logit.fit(train_X,train_draw)

predicted_p = draw_logit.predict_proba(train_X)[:,1]
indices = predicted_p.argsort()
indices = indices[-np.sum(train_draw):]
train_X['draw_maybe'] = 0
train_X.iloc[indices,-1] = 1 

predicted_p = draw_logit.predict_proba(test_X)[:,1]
indices = predicted_p.argsort()
no_draw = int(np.average(train_draw)*test_draw.size)
indices = indices[-no_draw:]
test_X['draw_maybe'] = 0
test_X.iloc[indices,-1] = 1

In [116]:
def new_f1_score(y_true,y_pred):
    t_away = np.sum(y_true==-1)
    t_draw = np.sum(y_true==0)
    t_home = np.sum(y_true==1)
    
    p_away = np.sum(y_true==-1)
    p_draw = np.sum(y_true==0)
    p_home = np.sum(y_true==1)
    
    total = y_true.size
    
    return (p_away/t_away)*(p_draw/t_draw)*(p_home/t_home)
    

In [117]:
#SVC
train_X['B365H'] = np.log(train_X['B365H']-1)
train_X['B365D'] = np.log(train_X['B365D']-1)

test_X['B365H'] = np.log(test_X['B365H'])
test_X['B365D'] = np.log(test_X['B365D'])

train_X = scale(train_X)
test_X = scale(test_X)

parameter_candidates = [{'C':[10,100,1000], 
                         'gamma': [0.0001,0.001,0.01]}]


clf = GridSearchCV(SVC(probability=True),parameter_candidates,scoring=make_scorer(new_f1_score),n_jobs=-1)
clf = GridSearchCV(SVC(probability=True),parameter_candidates,n_jobs=-1)

clf.fit(train_X,train_y)
svc = clf.best_estimator_
svc.fit(train_X,train_y)
print(svc.score(test_X,test_y))

0.610526315789


In [118]:
confusion_matrix(test_y,svc.predict(test_X))

array([[ 61,   0,  48],
       [ 12,   0,  72],
       [ 16,   0, 171]])

In [84]:
#Predicting draws: The ULTIMATE CHALLENGE (okay)
train_draw = np.array((train_y==0)+0)
test_draw = np.array((test_y==0)+0)
draw_logit = GaussianNB()
draw_logit.fit(train_X,train_draw)
multiplier = 1
p_draw = test_draw.size*(1-multiplier*np.average(train_draw))

predicted_p = draw_logit.predict_proba(test_X)[:,1]
indices = predicted_p.argsort()
indices = indices[int(p_draw):]
prediction = voter.predict(test_X)
print('Before: ', np.average(prediction==test_y))
prediction[indices] = 0
print('After: ', np.average(prediction==test_y))

Before:  0.610526315789
After:  0.597368421053


In [95]:
voter = svc

In [96]:
#Set-up for betting
odds = np.array(df16.iloc[:,1:4])
results = np.array(test_y)

In [98]:
#Betting strategy 1: Only bet if you're sure of winning (p)
p = voter.predict_proba(test_X)
pick = voter.predict(test_X)

#Replace home/away with draws from above
pick[indices] = 0

picked_odds = odds[np.arange(380),-pick+1]
profits = np.where(pick==results,picked_odds-1,-1)

threshold = 0
bets = (np.max(p,axis=1)-threshold)
bets[np.max(p,axis=1)<threshold] = 0
bets[bets!=0] = 1

profits = profits*bets
profit = np.sum(profits)/np.sum(bets)

print('Profit: ', profit)
print('Bets used: ', len(bets[bets!=0])/380)

Profit:  0.131868421053
Bets used:  1.0


In [96]:
#Betting strategy 2: Only bet when it's worth it (p*odds) (p from logit)

new_train_y = (train_y==voter.predict(train_X))*1

logit2 = LogisticRegressionCV(n_jobs=-1)
logit2.fit(train_X,new_train_y)


p = logit2.predict_proba(test_X)[:,1]


pick = voter.predict(test_X)
pick[indices] = 0

picked_odds = odds[np.arange(pick.size),-pick+1]
profits = np.where(pick==results,picked_odds-1,-1)

threshold = 0.5
expected = p*picked_odds-(1-p)
bets = np.where(expected>threshold,1.,0)
#bets *= np.abs(expected-threshold)

profits = profits*bets
profit = np.sum(profits)/np.sum(bets)

print('Profit: ', profit)
print('Bets used: ', len(bets[bets!=0])/380)

Profit:  0.129217687075
Bets used:  0.7736842105263158


In [90]:
#Betting strategy 3: Bet on the most profitable (using logit) outcome but only if over a threshold
#lol this is stupid
p = logit.predict_proba(test_X)
pick = np.argmax(p*odds-(1-p),axis=1)
expected = (p*odds-(1-p))[np.arange(results.size),pick]
picked_odds = odds[np.arange(results.size),pick]

threshold = 0.9
bets = np.where(expected>threshold,1,0)

profits = np.where(results==-pick+1,picked_odds-1,-1)

profit = np.sum(profits)/np.sum(bets)

print('Profit: ', profit)
print('Bets used: ', len(bets[bets!=0])/380)

Profit:  -0.264455782313
Bets used:  0.7736842105263158


In [91]:
#Betting strategy 4: Always bet home (yes he did the absolute legend)
profits = np.where(results==1,odds[:,0]-1,-1)
profit = np.sum(profits)/len(test_X)

print('Profit: ', profit)
print('Bets used: ', 1)

Profit:  0.0656052631579
Bets used:  1


In [92]:
#Betting strategy 5: Only bet when it's worth it (using the confusion matrix)

pick = voter.predict(test_X)
picked_odds = odds[np.arange(pick.size),-pick+1]
profits = np.where(pick==results,picked_odds-1,-1)

cross_pick = np.array(cross_val_predict(voter,train_X,train_y))
actual_results = np.array(train_y)

mtrx = confusion_matrix(actual_results,cross_pick)
p = np.copy(pick).astype('float')
p[pick==-1] = mtrx[0,0]/np.sum(mtrx[0,:])
p[pick==1] = mtrx[2,2]/np.sum(mtrx[2,:])
p[pick==0] = mtrx[1,1]/np.sum(mtrx[1,:])

threshold = 1
expected = p*picked_odds-(1-p)
bets = np.where(expected>threshold,1.,0)
#bets *= np.abs(expected-threshold)

profits = profits*bets
profit = np.sum(profits)/np.sum(bets)

print('Profit: ', profit)
print('Bets used: ', len(bets[bets!=0])/380)

Profit:  0.127512437811
Bets used:  0.5289473684210526


In [151]:
#Betting strategy 6: Bet randomly
M = 100
profit_ns = np.zeros(M)
for n in np.arange(M):
    pick = np.random.randint(-1,2,results.size)
    picked_odds = odds[np.arange(pick.size),-pick+1]
    profits = np.where(pick==results,picked_odds-1,-1)
    profit = np.sum(profits)/pick.size
    profit_ns[n] = profit
print('Mean: ', np.average(profit_ns))
print('Standard Deviation: ', np.std(profit_ns))

Mean:  -0.0820757894737
Standard Deviation:  0.0765888405872


In [150]:
pick

array([ 0,  1,  0,  1, -1,  0,  0,  1,  0,  0,  1,  1,  0,  0, -1,  0,  0,
       -1, -1,  0,  0, -1,  1, -1,  0,  1,  1,  0,  0,  1,  1, -1,  1,  0,
        1,  0, -1,  0,  1,  1,  0,  0,  1, -1, -1,  1,  0, -1,  0,  1,  1,
        0,  0,  0,  1,  0,  1, -1, -1,  1,  1, -1, -1,  1,  1,  1, -1,  0,
        0,  1,  0,  0,  1,  1,  1, -1,  0,  1,  0,  0, -1,  0, -1,  0,  0,
       -1,  0,  0,  0,  1, -1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  0,
        0,  1, -1, -1, -1, -1,  1,  1, -1,  0,  1,  0, -1, -1,  0, -1, -1,
        1, -1,  0,  0, -1,  0,  1,  1,  1,  1,  0,  1,  1,  1,  1,  1, -1,
        1,  1, -1,  1, -1,  1,  1,  1, -1,  1,  0,  0, -1,  1, -1,  0, -1,
        1,  0,  0, -1, -1,  1,  0,  1,  1, -1,  0,  0,  1,  1, -1,  0,  0,
       -1, -1, -1,  1,  0,  0, -1,  1, -1, -1, -1,  1, -1,  0,  1, -1, -1,
       -1,  0, -1,  0,  1, -1, -1,  0,  0,  0,  1,  0,  1, -1,  0,  1,  1,
        0,  0,  0,  1, -1, -1, -1,  0,  0,  0, -1,  1,  1, -1,  1, -1,  1,
        0,  1, -1, -1, -1

In [104]:
mtrx = confusion_matrix(results,pick)

TypeError: Expected sequence or array-like, got <class 'int'>

In [74]:
labels = ['Away', 'Draw', 'Home']
sns.heatmap(mtrx,annot=True,xticklabels=labels,yticklabels=labels)
plt.show()

In [205]:
#Replace 0 with 0 mtrx[0,0]/np.sum(mtrx[0,:])
ps = np.random.rand(len(test_y))
mtrx[0,0]/np.sum(mtrx[0,:])
initial_pred = voter.predict(test_X)+1
final_pred = np.zeros(initial_pred.shape)
proportion = 1



for n in np.arange(3):
    every = np.arange(initial_pred.size)
    ind1 = np.where(initial_pred==n) #Initial prediction is n
    ind2 = np.where(ps<=mtrx[n,0]/np.sum([mtrx[n,:]])) #Probability is less than the first threshold
    ind3 = np.where(ps<=(mtrx[n,0]+mtrx[n,1])/np.sum([mtrx[n,:]])) #Probability is less than the second threshold
    ind4 = np.where(ps>(mtrx[n,0]+mtrx[n,1])/np.sum([mtrx[n,:]])) #Probability is less than the second threshold
    
    
    
    
    final_pred[np.intersect1d(ind1,ind3)] = 1
    final_pred[np.intersect1d(ind1,ind2)] = 0
    final_pred[np.intersect1d(ind1,ind4)] = 2



In [206]:
confusion_matrix(test_y+1,final_pred)

array([[ 57,   0,  52],
       [ 27,   0,  57],
       [ 45,   0, 142]])

In [208]:
np.sum(final_pred==test_y+1)/380

0.52368421052631575

In [209]:
train_X

,stage,B365H,B365D,B365A,competence_difference,draw_maybe
0,1,1.67,3.60,5.50,0.000000,0
1,1,3.20,3.25,2.30,0.000000,1
2,1,2.63,3.30,2.70,0.000000,1
3,1,3.60,3.25,2.10,0.000000,1
4,1,2.25,3.25,3.25,0.000000,1
5,1,2.63,3.20,2.75,0.000000,1
6,1,1.91,3.30,4.33,0.000000,1
7,1,1.17,6.50,21.00,0.000000,0
8,1,1.20,6.00,17.00,0.000000,0
9,1,3.20,3.25,2.30,0.000000,1
